In [1]:
import numpy as np
from operator import itemgetter
import networkx as nx
import matplotlib.pyplot as plt
from gurobipy import *

# Data

In [2]:
# An array of distances between customers.
dist = np.array([
    [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
    [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
    [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
    [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
    [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
    [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
    [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
    [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
    [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
    [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
    [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
    [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
    [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
    [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
    [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
    [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
    [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0]
])

# Each location has a demand corresponding to the quantity—for example, weight
# or volume—of the item to be delivered.
dem = np.array([0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8])

# The number of vehicles in the fleet.
n_vehicles = 5

# Each vehicle has a capacity: the maximum quantity that it can hold.
cap = 15

In [3]:
assert dist.shape[0]==dist.shape[1]==len(dem), 'Check input data dimensions'
assert dem[0]==0, 'Check demand data'

In [4]:
n_customers = len(dem)-1

# MIP with Gurobi

In [5]:
def subtour_elim(model, where):
    
    # Found a new (integer) solution to pricing problem problem, so use callback
    if where == GRB.callback.MIPSOL:
        
        # Extract 'x' values from solution
        x_sol = np.array(model.cbGetSolution(model._vars)).reshape(n_customers+1, n_customers+1, n_vehicles)
        x_sol = np.where(x_sol>0.5, 1, 0)  # Account for solver Numerical stability
        
        # Transform solution into graph representation
        G_sol = []
        for u in range(n_vehicles):
            G_sol.append( nx.DiGraph(x_sol[:,:,u]) )
        
        for u in range(n_vehicles):
            for cycle in nx.simple_cycles(G_sol[u]):
                if cycle[0] != 0:
                    subtour = cycle
                    subtour.append(subtour[0])
                    
                    # Set-up subtour elimination constraint
                    expr = LinExpr()
                    for i in range(len(subtour)-1):
                        expr += x[subtour[i], subtour[i+1], u]
                    model.cbLazy(expr <= len(subtour)-2)

In [6]:
# Initialize problem instance
cvrp = Model('cvrp_problem')

# Add binary variables
x = cvrp.addVars(n_customers+1, n_customers+1, n_vehicles, vtype=GRB.BINARY, name='x')
cvrp.update()

# Add constraints
for u in range(n_vehicles):
    cvrp.addConstr(quicksum(dem[i]*x.sum(i,'*',u) for i in range(n_customers+1)) <= cap,
                      name='capacity_constr_'+str(u))
    cvrp.addConstr(quicksum(x.sum(0,j,u) for j in range(n_customers+1)) <= 1,
                      name='depot_out_constr_'+str(u))
    for k in range(n_customers+1):
        cvrp.addConstr(x.sum('*',k,u) == x.sum(k,'*',u),
                          name='vertex_in_out_constr_'+str(k)+'_'+str(u))
    for l in range(1, n_customers+1):
        cvrp.addConstr(x[l,l,u] == 0,
                          name='prevent_self_loop_constr_'+str(k)+'_'+str(u))

for i in range(1, n_customers+1):
    cvrp.addConstr(quicksum(x.sum(i,'*',u) for u in range(n_vehicles)) == 1,
                      name='customer_visit_constr'+str(i))

# Initialize objective function
obj_cvrp = LinExpr()
for u in range(n_vehicles):
    for i in range(n_customers+1):
        for j in range(n_customers+1):
            obj_cvrp += dist[i,j]*x[i,j,u]

# Set objective
cvrp.setObjective(obj_cvrp, GRB.MINIMIZE)

# Gurobi params
cvrp.setParam('MIPGapAbs', 620)  # Terminate when MIP gap < 10% (=620)
cvrp.setParam('MIPFocus', 2)     # Work towards proving optimality
cvrp.setParam('LazyConstraints', 1)

# Solve cvrp problem
cvrp._vars = cvrp.getVars()
cvrp.optimize(subtour_elim)

Academic license - for non-commercial use only
Changed value of parameter MIPGapAbs to 620.0
   Prev: 1e-10  Min: 0.0  Max: 1e+100  Default: 1e-10
Changed value of parameter MIPFocus to 2
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Optimize a model with 191 rows, 1445 columns and 5605 nonzeros
Variable types: 0 continuous, 1445 integer (1445 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+02, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 80 rows and 140 columns
Presolve time: 0.05s
Presolved: 111 rows, 1305 columns, 5125 nonzeros
Variable types: 0 continuous, 1305 integer (1305 binary)
Presolve removed 5 rows and 0 columns
Presolved: 106 rows, 1305 columns, 4995 nonzeros


Root relaxation: objective 3.880000e+03, 50 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl 

 379825 231965 6164.00000  117   18 6208.00000 4975.46667  19.9%  19.7  415s
 383729 233900 5587.06667   53   52 6208.00000 4977.77293  19.8%  19.7  420s
 387751 235845 5519.11111   44   28 6208.00000 4980.70238  19.8%  19.7  425s
 391829 237803     cutoff   85      6208.00000 4983.68794  19.7%  19.8  430s
 395473 239588 5292.96861   48   72 6208.00000 4986.66667  19.7%  19.8  435s
 399645 241819     cutoff   74      6208.00000 4989.41053  19.6%  19.8  440s
 403508 243761 infeasible  109      6208.00000 4992.30769  19.6%  19.8  445s
 406470 245244 6055.95291   84   54 6208.00000 4994.84249  19.5%  19.8  450s
 410710 247266 5669.50000   57   35 6208.00000 4997.84615  19.5%  19.9  455s
 414688 249143 6143.11111   90   34 6208.00000 5000.00000  19.5%  19.9  460s
 418573 251074     cutoff   57      6208.00000 5002.09611  19.4%  19.9  465s
 422403 252984 5153.70588   56   38 6208.00000 5004.76923  19.4%  19.9  470s
 426748 255024 5929.40000   38   36 6208.00000 5007.98281  19.3%  20.0  475s

 757452 400575 6098.00000   64   27 6208.00000 5175.40000  16.6%  21.4  950s
 760640 401911 5699.00000   57   34 6208.00000 5176.74751  16.6%  21.4  955s
 763852 403213 5712.62572  101   60 6208.00000 5178.21739  16.6%  21.4  960s
 767068 404486 5967.78378   61   52 6208.00000 5179.82353  16.6%  21.5  965s
 770825 405954 5788.17000   49   60 6208.00000 5181.24395  16.5%  21.5  970s
 773144 406873     cutoff   42      6208.00000 5182.00000  16.5%  21.5  975s
 776430 408183 6056.00000   54    6 6208.00000 5183.21429  16.5%  21.5  980s
 779239 409353 5893.66071   77   59 6208.00000 5184.20000  16.5%  21.5  985s
 781863 410410 6017.40845   74   53 6208.00000 5185.42857  16.5%  21.5  990s
 785737 411870 6171.76744   99   45 6208.00000 5187.09333  16.4%  21.5  995s
 789022 413164 5598.66667   52   41 6208.00000 5188.44444  16.4%  21.5 1000s
 791881 414337 5491.73856   42   54 6208.00000 5189.50000  16.4%  21.5 1005s
 795608 415813 5977.90232   86   78 6208.00000 5191.12941  16.4%  21.5 1010s

 1082335 519593 5807.23355   87   51 6208.00000 5282.56277  14.9%  22.3 1485s
 1085606 520732 6081.72477   73   53 6208.00000 5283.33333  14.9%  22.3 1490s
 1088535 521713     cutoff   78      6208.00000 5284.00000  14.9%  22.3 1495s
 1091451 522716 5869.32630   78   57 6208.00000 5284.79732  14.9%  22.3 1500s
 1094673 523895 5664.46154   50   77 6208.00000 5285.68320  14.9%  22.3 1505s
 1097534 524818 5815.22807   71   29 6208.00000 5286.55258  14.8%  22.3 1511s
 1099540 525424 5973.76137   76   62 6208.00000 5287.02418  14.8%  22.3 1515s
 1102450 526426 5846.33786   65   78 6208.00000 5287.85714  14.8%  22.3 1520s
 1105770 527477     cutoff   51      6208.00000 5288.91752  14.8%  22.3 1525s
 1108635 528412     cutoff   42      6208.00000 5289.83333  14.8%  22.3 1530s
 1111450 529339 5868.32692  104   58 6208.00000 5290.66667  14.8%  22.3 1535s
 1114700 530423 infeasible   84      6208.00000 5291.73333  14.8%  22.3 1540s
 1117695 531415 6152.66667   53   31 6208.00000 5292.42424  14.7

 1369681 610045 5800.16667   53   35 6208.00000 5355.72927  13.7%  22.8 2015s
 1372443 610874 5765.04673   66   36 6208.00000 5356.35294  13.7%  22.8 2020s
 1374765 611665 5982.00000   38   35 6208.00000 5356.93333  13.7%  22.9 2025s
 1377550 612570 6023.15498   56   52 6208.00000 5357.54054  13.7%  22.9 2030s
 1379570 613165 5972.33333   52   57 6208.00000 5358.00000  13.7%  22.9 2035s
 1381938 613781 5670.33333   35   33 6208.00000 5358.59361  13.7%  22.9 2040s
 1384262 614470     cutoff   56      6208.00000 5358.99320  13.7%  22.9 2045s
 1386469 615121 6128.76522   87   48 6208.00000 5359.63636  13.7%  22.9 2050s
 1387936 615526 6054.40000   49   25 6208.00000 5360.00000  13.7%  22.9 2055s
 1389394 616005     cutoff   60      6208.00000 5360.33333  13.7%  22.9 2060s
 1392363 616805 5761.44000   52   44 6208.00000 5360.90909  13.6%  22.9 2065s
 1394737 617525 5965.18904   72   50 6208.00000 5361.48805  13.6%  22.9 2070s
 1397544 618375 6043.00000   72   50 6208.00000 5362.10343  13.6

...

 1820006 724689 5986.76795   51   80 6208.00000 5448.72500  12.2%  23.7 3075s
 1821903 725049 6044.25420   52   58 6208.00000 5449.11111  12.2%  23.7 3080s
 1824268 725582 6163.45121   31   60 6208.00000 5449.51667  12.2%  23.7 3085s
 1826633 726108 5768.14108   92   84 6208.00000 5450.00000  12.2%  23.7 3090s
 1828548 726599 5844.06993   47   76 6208.00000 5450.34014  12.2%  23.7 3095s
 1830484 727029     cutoff   85      6208.00000 5450.66667  12.2%  23.7 3100s
 1832319 727506 6017.33333   50   32 6208.00000 5450.97674  12.2%  23.7 3105s
 1834196 727904 6037.75902  118   67 6208.00000 5451.32174  12.2%  23.7 3110s
 1836094 728347 6206.00000   45   24 6208.00000 5451.68137  12.2%  23.7 3115s
 1838110 728765 5782.40293   46   44 6208.00000 5452.00000  12.2%  23.7 3120s
 1840047 729213 6176.84388   69   58 6208.00000 5452.38333  12.2%  23.7 3125s
 1842391 729747 5844.00000   52    8 6208.00000 5452.77785  12.2%  23.7 3131s
 1842859 729848 6188.00000   55   14 6208.00000 5452.86957  12.2

 2029547 768418 6101.00000   59   24 6208.00000 5485.14286  11.6%  24.0 3605s
 2031984 768894 5880.62500   46   34 6208.00000 5485.57252  11.6%  24.0 3610s
 2033903 769243     cutoff   98      6208.00000 5485.87302  11.6%  24.0 3615s
 2036115 769627 6130.80000   82   16 6208.00000 5486.32488  11.6%  24.0 3620s
 2038607 770090 6036.63158   58   31 6208.00000 5486.70874  11.6%  24.0 3625s
 2041054 770607 5623.65803   77   81 6208.00000 5487.16667  11.6%  24.0 3632s
 2041518 770649 infeasible   42      6208.00000 5487.25000  11.6%  24.0 3635s
 2043446 771069 6098.39676   80   46 6208.00000 5487.63543  11.6%  24.0 3640s
 2045323 771487 6113.42857   93   29 6208.00000 5487.96889  11.6%  24.0 3645s
 2047830 771986 6109.33333   49   45 6208.00000 5488.25714  11.6%  24.0 3650s
 2049575 772359 6106.38253   75   59 6208.00000 5488.54545  11.6%  24.0 3655s
 2051968 772735 5755.33333   51   29 6208.00000 5489.00000  11.6%  24.0 3660s
 2054240 773219 6075.42857   68   47 6208.00000 5489.33333  11.6

 2242141 808132 6172.00000   54   14 6208.00000 5518.83673  11.1%  24.1 4135s
 2243710 808363 5762.74699   35   56 6208.00000 5519.06061  11.1%  24.2 4140s
 2245606 808671     cutoff   89      6208.00000 5519.38462  11.1%  24.2 4145s
 2247511 809043 6165.00000   48   32 6208.00000 5519.69293  11.1%  24.2 4150s
 2250002 809486 6201.33333   38   23 6208.00000 5520.02978  11.1%  24.2 4156s
 2251843 809875 infeasible   52      6208.00000 5520.33333  11.1%  24.2 4160s
 2254214 810238 infeasible  110      6208.00000 5520.69091  11.1%  24.2 4166s
 2256119 810603 5797.00000   64   42 6208.00000 5521.00000  11.1%  24.2 4170s
 2257130 810755 5906.60000   74   49 6208.00000 5521.10638  11.1%  24.2 4176s
 2258663 810966 6122.20408   64   30 6208.00000 5521.33333  11.1%  24.2 4180s
 2260590 811289 5931.30333   82   87 6208.00000 5521.61290  11.1%  24.2 4185s
 2262471 811606 5976.75758  105   29 6208.00000 5521.92325  11.1%  24.2 4190s
 2264455 811899 6078.18408   33   37 6208.00000 5522.17730  11.0

 2457413 844111 6088.70270   59   32 6208.00000 5549.33333  10.6%  24.3 4665s
 2459404 844332 6036.00000   65   16 6208.00000 5549.66667  10.6%  24.3 4670s
 2461837 844698 6116.93894   45   50 6208.00000 5550.00000  10.6%  24.3 4675s
 2463809 845058 6011.57143   42   65 6208.00000 5550.22754  10.6%  24.3 4680s
 2465650 845339 5895.08536   64   82 6208.00000 5550.52879  10.6%  24.3 4685s
 2467488 845589 6127.09091   68   39 6208.00000 5550.80734  10.6%  24.3 4690s
 2469789 846043 6066.52323  111   81 6208.00000 5551.11111  10.6%  24.3 4695s
 2472145 846423 5740.03941   50   44 6208.00000 5551.44000  10.6%  24.3 4700s
 2474106 846663 6170.57143   60   66 6208.00000 5551.69231  10.6%  24.3 4705s
 2476516 847041     cutoff   82      6208.00000 5552.02931  10.6%  24.3 4711s
 2478355 847377     cutoff   43      6208.00000 5552.33914  10.6%  24.3 4717s
 2479335 847448 6126.08000   45   30 6208.00000 5552.50000  10.6%  24.3 4720s
 2481705 847744 6131.93455   94   50 6208.00000 5552.85714  10.6

 2670079 874154 6013.36986   93   31 6208.00000 5578.47423  10.1%  24.4 5195s
 2672373 874487 6050.00000   59   31 6208.00000 5578.79853  10.1%  24.4 5201s
 2673842 874687 6009.49409   85   40 6208.00000 5578.97587  10.1%  24.4 5205s
 2675795 874997 6086.00000   86   44 6208.00000 5579.20514  10.1%  24.4 5210s
 2678152 875273     cutoff   72      6208.00000 5579.50000  10.1%  24.4 5215s
 2680044 875614 5780.35961  109   71 6208.00000 5579.77778  10.1%  24.4 5220s
 2680365 875642     cutoff   89      6208.00000 5579.81818  10.1%  24.4 5225s
 2682469 875882 5944.88889   58   50 6208.00000 5580.00000  10.1%  24.4 5230s
 2684908 876220 6098.92586   88   48 6208.00000 5580.32000  10.1%  24.4 5235s
 2686765 876482 6152.96970   81   54 6208.00000 5580.53333  10.1%  24.4 5241s
 2688198 876634     cutoff   53      6208.00000 5580.75000  10.1%  24.4 5245s
 2690164 876860 6115.20690   71   45 6208.00000 5581.00000  10.1%  24.4 5250s
 2692038 877106 5816.00000   76   25 6208.00000 5581.26667  10.1

### View solution

In [10]:
x_sol = np.array(cvrp.X).reshape(n_customers+1, n_customers+1, n_vehicles)
x_sol = np.where(x_sol>0.5, 1, 0)

G_sol = []
for u in range(n_vehicles):
    G_sol.append( nx.DiGraph(x_sol[:,:,u]) )

for u in range(n_vehicles):
    for route in nx.simple_cycles(G_sol[u]):
        print(route)

[0, 12, 11, 15, 13]
[0, 5, 6, 2, 8]
[0, 1, 3, 4, 7]
[0, 9, 10, 16, 14]


### While vanilla MIP formulation quickly finds the optimal solution for this instance (17 seconds), it takes a lot of time to prove optimality. We're only able to converge to 10% gap after ~1.5 hours.